In [1]:
import json, argparse, sys
import pandas as pd
sys.path.append("../src")

In [6]:
import torch
from baseline.src.main import *
import yaml

type_data = 'valid'

config_path = '../finetuned_llm/bioserc_bert_based/version_13/hparams.yaml'
model_path = "../finetuned_llm/bioserc_bert_based/roberta-large-meld-valid/f1=67.41.ckpt"

print('config_path',config_path)

with open(config_path, "r") as yamlfile:
    model_configs = argparse.Namespace(**yaml.load(yamlfile, Loader=yaml.FullLoader))
    
model_configs.data_folder = '../data/'
# model_configs.window_ct = 2
# model_configs.speaker_description = False
# model_configs.llm_context = False
# model_configs.data_name_pattern = "meld.{}.json"
dataset_name = model_configs.data_name_pattern.split(".")[0]


config_path ../finetuned_llm/bioserc_bert_based/version_13/hparams.yaml


In [7]:
print(dataset_name)

meld


In [8]:


# meld
label2id = {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger':6}
id2label = ['neutral', 'surprise', 'fear', 'sadness', 'joy', 'disgust', 'anger']


# # emorynlp
# label2id = {'Joyful': 0, 'Mad': 1, 'Peaceful': 2, 'Neutral': 3, 'Sad': 4, 'Powerful': 5, 'Scared': 6}
# id2label = ['Joyful', 'Mad', 'Peaceful', 'Neutral', 'Sad', 'Powerful', 'Scared']
# # iemocap
# label2id = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5}
# id2label = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']


In [9]:

from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel
import json
import random
import argparse
from sklearn.metrics import f1_score

import pytorch_lightning as pl
from pytorch_lightning import Trainer

bert_tokenizer = AutoTokenizer.from_pretrained(model_configs.pre_trained_model_name)

data_loader_valid = BatchPreprocessor(bert_tokenizer, model_configs=model_configs, data_type=type_data)
raw_data = BatchPreprocessor.load_raw_data(f"{model_configs.data_folder}/{model_configs.data_name_pattern.format(type_data)}")
valid_loader = DataLoader(raw_data,
                          batch_size=model_configs.batch_size, collate_fn=data_loader_valid, shuffle=False)

data_loader_test=BatchPreprocessor(bert_tokenizer, model_configs=model_configs, data_type='test')
raw_data_test = BatchPreprocessor.load_raw_data(f"{model_configs.data_folder}/{model_configs.data_name_pattern.format('test')}")
test_loader = DataLoader(raw_data_test,
                          batch_size=model_configs.batch_size, collate_fn=data_loader_test, shuffle=False)


In [10]:
model_configs.spdesc_aggregate_method = 'static'
# model_configs.llm_context = False
# model_configs.speaker_description=False
model = EmotionClassifier(model_configs) 
# model.model_configs = model_configs

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:

import json 
import itertools 
model = EmotionClassifier.load_from_checkpoint(model_path, strict=False, model_configs=model_configs)
trainer = Trainer(max_epochs=1,  accelerator="gpu", devices=1,  )

print(trainer.test(model, valid_loader))
print(trainer.test(model, test_loader))


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /home/phuongnm/BiosERC/scripts/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-534884ef-a89c-23e9-885a-13ca043a4659,GPU-e2bff860-9659-2d80-4f5d-b02e78499047]
/home/phuongnm

Testing: |                                                                                                    …

/home/phuongnm/py_envs/python/env_llm/lib/python3.9/site-packages/torch/nn/modules/transformer.py:384: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:177.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)
/home/phuongnm/py_envs/python/env_llm/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/home/phuongnm/py_envs/python/env_llm/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 23. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/home/phuongnm/py_envs/python/env_llm/lib/python3.9/site-packages/pytorc

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         hp_metric         │     67.41372680664062     │
│          test/f1          │     67.41372680664062     │
│        train/loss         │    1.7146971225738525     │
│        valid/loss         │    1.7146971225738525     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-534884ef-a89c-23e9-885a-13ca043a4659,GPU-e2bff860-9659-2d80-4f5d-b02e78499047]


...

/home/phuongnm/py_envs/python/env_llm/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=51` in the `DataLoader` to improve performance.


Testing: |                                                                                                    …

/home/phuongnm/py_envs/python/env_llm/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 24. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/home/phuongnm/py_envs/python/env_llm/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 40. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/home/phuongnm/py_envs/python/env_llm/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 25. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/home/phuongnm/py_envs/python/env_llm/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         hp_metric         │     65.95140075683594     │
│          test/f1          │     65.95140075683594     │
│        train/loss         │    1.7551467418670654     │
│        valid/loss         │    1.7551467418670654     │
└───────────────────────────┴───────────────────────────┘

[{'train/loss': 1.7551467418670654, 'valid/loss': 1.7551467418670654, 'test/f1': 65.95140075683594, 'hp_metric': 65.95140075683594}]
